In [3]:
# Install required packages
# pip install ultralytics opencv-python numpy matplotlib

from ultralytics import YOLO
import cv2
import numpy as np
import os
import csv

# ============================================================================
# FUNCTION: Calculate angle between three points
# ============================================================================
def calculate_angle(p1, p2, p3):
    """
    Calculate angle at point p2 formed by points p1-p2-p3
    
    Args:
        p1: First point (shoulder) - [x, y]
        p2: Middle point (elbow) - [x, y] 
        p3: Third point (wrist) - [x, y]
    
    Returns:
        Angle in degrees
    """
    p1 = np.array(p1)
    p2 = np.array(p2)
    p3 = np.array(p3)
    
    # Create vectors from elbow to shoulder and elbow to wrist
    vector1 = p1 - p2  # shoulder to elbow
    vector2 = p3 - p2  # wrist to elbow
    
    # Calculate dot product and magnitudes
    dot_product = np.dot(vector1, vector2)
    magnitude1 = np.linalg.norm(vector1)
    magnitude2 = np.linalg.norm(vector2)
    
    # Avoid division by zero
    if magnitude1 == 0 or magnitude2 == 0:
        return 0.0
    
    # Calculate cosine of angle
    cosine_angle = dot_product / (magnitude1 * magnitude2)
    
    # Clamp to [-1, 1] to avoid numerical errors
    cosine_angle = np.clip(cosine_angle, -1.0, 1.0)
    
    # Calculate angle in radians then convert to degrees
    angle_radians = np.arccos(cosine_angle)
    angle_degrees = np.degrees(angle_radians)
    
    return angle_degrees

# ============================================================================
# FUNCTION: Select which person to analyze (left or right view)
# ============================================================================
def select_person_by_position(keypoints, position='left'):
    """
    Select person based on their horizontal position in frame
    
    Args:
        keypoints: Array of detected keypoints [num_people, 17, 2]
        position: 'left' for leftmost person, 'right' for rightmost, 
                  'center' for middle person
    
    Returns:
        Index of selected person
    """
    if len(keypoints) == 0:
        return None
    
    if len(keypoints) == 1:
        return 0
    
    # Calculate average x-coordinate for each person (to find their position)
    avg_x_positions = []
    for person_kp in keypoints:
        # Use nose, shoulders, and hips to determine position
        valid_points = person_kp[[0, 5, 6, 11, 12]]  # nose, shoulders, hips
        valid_x = valid_points[:, 0][valid_points[:, 0] > 0]  # filter out zero coordinates
        if len(valid_x) > 0:
            avg_x_positions.append(np.mean(valid_x))
        else:
            avg_x_positions.append(0)
    
    # Select based on position
    if position == 'left':
        return np.argmin(avg_x_positions)  # Leftmost person
    elif position == 'right':
        return np.argmax(avg_x_positions)  # Rightmost person
    elif position == 'center':
        return len(keypoints) // 2  # Middle person
    else:
        return 0  # Default to first detection

# ============================================================================
# CONFIGURATION
# ============================================================================

# Load YOLOv8-Pose model
model = YOLO('yolov8n-pose.pt')

# YOLOv8 Keypoint indices (COCO format)
LEFT_SHOULDER = 5
RIGHT_SHOULDER = 6
LEFT_ELBOW = 7
RIGHT_ELBOW = 8
LEFT_WRIST = 9
RIGHT_WRIST = 10

# ⚠️ FIXED: Frame files with SPACES (not hyphens) and .png extension
frame_files = ['Frame 1.png', 'Frame 2.png', 'Frame 3.png', 
               'Frame 4.png', 'Frame 5.png', 'Frame 6.png', 'Frame 7.png']

# IMPORTANT: Choose which person view to analyze
# 'left' = left side of frame, 'right' = right side of frame
PERSON_POSITION = 'left'  # Change to 'right' to analyze the right view

# Create output directories
os.makedirs('pose_output', exist_ok=True)
os.makedirs('angle_output', exist_ok=True)

print("=" * 80)
print("GOLF SWING POSE ANALYSIS - Tiger Woods")
print("=" * 80)
print(f"📍 Analyzing: {PERSON_POSITION.upper()} person in frame")
print("=" * 80)

# ============================================================================
# STEP 1: POSE ESTIMATION ON ALL FRAMES
# ============================================================================

all_results = []

for frame_file in frame_files:
    # Read frame
    img = cv2.imread(frame_file)
    
    if img is None:
        print(f"❌ Could not read {frame_file}")
        print(f"   Current working directory: {os.getcwd()}")
        print(f"   Looking for file: {os.path.abspath(frame_file)}")
        continue
    
    # Run pose estimation
    results = model(img, verbose=False)
    
    # Get keypoints
    keypoints = results[0].keypoints.xy.cpu().numpy()
    keypoints_conf = results[0].keypoints.conf.cpu().numpy()
    
    num_people = len(keypoints)
    print(f"\n{frame_file}:")
    print(f"  Detected {num_people} person(s)")
    
    if num_people > 0:
        # Select which person to analyze
        person_idx = select_person_by_position(keypoints, PERSON_POSITION)
        
        if person_idx is not None:
            kp = keypoints[person_idx]
            conf = keypoints_conf[person_idx]
            
            print(f"  Selected person at index {person_idx} ({PERSON_POSITION} position)")
            print(f"  Average keypoint confidence: {conf.mean():.3f}")
            
            all_results.append({
                'frame': frame_file,
                'keypoints': kp,
                'confidence': conf,
                'num_people': num_people,
                'person_idx': person_idx
            })
            
            # Save pose-overlaid image (showing all detections)
            annotated_img = results[0].plot()
            
            # Highlight selected person with text
            cv2.putText(annotated_img, f"Analyzing {PERSON_POSITION.upper()} person", 
                       (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)
            
            cv2.imwrite(f'pose_output/pose_{frame_file}', annotated_img)
            print(f"  ✓ Saved pose overlay")

print(f"\n✓ Processed {len(all_results)} frames successfully")

# ============================================================================
# STEP 2: CALCULATE ARM ANGLES FOR ALL FRAMES
# ============================================================================

print("\n" + "=" * 80)
print("ARM ANGLE CALCULATION")
print("=" * 80)

results_data = []

for i, result in enumerate(all_results, 1):
    kp = result['keypoints']
    conf = result['confidence']
    frame_file = result['frame']
    
    # Extract arm keypoints
    right_shoulder = kp[RIGHT_SHOULDER]
    right_elbow = kp[RIGHT_ELBOW]
    right_wrist = kp[RIGHT_WRIST]
    
    left_shoulder = kp[LEFT_SHOULDER]
    left_elbow = kp[LEFT_ELBOW]
    left_wrist = kp[LEFT_WRIST]
    
    # Calculate angles
    right_arm_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
    left_arm_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    
    # Store results
    results_data.append({
        'frame': frame_file,
        'frame_num': i,
        'right_angle': right_arm_angle,
        'left_angle': left_arm_angle,
        'right_conf': (conf[RIGHT_SHOULDER] + conf[RIGHT_ELBOW] + conf[RIGHT_WRIST]) / 3,
        'left_conf': (conf[LEFT_SHOULDER] + conf[LEFT_ELBOW] + conf[LEFT_WRIST]) / 3
    })
    
    print(f"\n📊 Frame {i}: {frame_file}")
    print(f"   Right Arm Angle: {right_arm_angle:.1f}°")
    print(f"   Left Arm Angle:  {left_arm_angle:.1f}°")
    
    # Load and annotate image with angles
    img = cv2.imread(frame_file)
    results_viz = model(img, verbose=False)
    annotated_img = results_viz[0].plot()
    
    # Add angle text
    cv2.putText(annotated_img, f"Right Arm: {right_arm_angle:.1f}deg", 
                (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
    cv2.putText(annotated_img, f"Left Arm: {left_arm_angle:.1f}deg", 
                (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
    cv2.putText(annotated_img, f"Frame {i}", 
                (20, 150), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)
    
    cv2.imwrite(f'angle_output/angle_{frame_file}', annotated_img)

# ============================================================================
# STEP 3: SUMMARY TABLE
# ============================================================================

print("\n" + "=" * 80)
print("SUMMARY TABLE - ARM ANGLES ACROSS ALL FRAMES")
print("=" * 80)
print(f"{'Frame':<15} {'Frame #':<10} {'Right Arm (°)':<15} {'Left Arm (°)':<15} {'Change (°)':<15}")
print("-" * 80)

for idx, data in enumerate(results_data):
    if idx == 0:
        change = "-"
    else:
        change_val = data['right_angle'] - results_data[idx-1]['right_angle']
        change = f"+{change_val:.1f}" if change_val > 0 else f"{change_val:.1f}"
    
    print(f"{data['frame']:<15} {data['frame_num']:<10} {data['right_angle']:>12.1f}   "
          f"{data['left_angle']:>12.1f}   {change:>12}")

print("=" * 80)

# ============================================================================
# STEP 4: COMPARE SPECIFIC FRAMES (User can modify these)
# ============================================================================

print("\n" + "=" * 80)
print("ARM ANGLE CHANGE BETWEEN SPECIFIC FRAMES")
print("=" * 80)

# USER INPUT: Choose which frames to compare (1-indexed)
frame1_idx = 1  # First frame to compare
frame2_idx = len(results_data)  # Last frame to compare

if frame1_idx <= len(results_data) and frame2_idx <= len(results_data):
    frame1_data = results_data[frame1_idx - 1]
    frame2_data = results_data[frame2_idx - 1]
    
    right_angle_change = frame2_data['right_angle'] - frame1_data['right_angle']
    left_angle_change = frame2_data['left_angle'] - frame1_data['left_angle']
    
    print(f"\n🔍 Comparing Frame {frame1_idx} ({frame1_data['frame']}) vs Frame {frame2_idx} ({frame2_data['frame']}):\n")
    print(f"   RIGHT ARM:")
    print(f"      Frame {frame1_idx}: {frame1_data['right_angle']:.1f}°")
    print(f"      Frame {frame2_idx}: {frame2_data['right_angle']:.1f}°")
    print(f"      Change: {right_angle_change:+.1f}° ({'extension' if right_angle_change > 0 else 'flexion'})")
    
    print(f"\n   LEFT ARM:")
    print(f"      Frame {frame1_idx}: {frame1_data['left_angle']:.1f}°")
    print(f"      Frame {frame2_idx}: {frame2_data['left_angle']:.1f}°")
    print(f"      Change: {left_angle_change:+.1f}° ({'extension' if left_angle_change > 0 else 'flexion'})")

# ============================================================================
# STEP 5: DELIVERABLES FOR REPORT
# ============================================================================

print("\n" + "=" * 80)
print("📋 DELIVERABLES FOR YOUR REPORT")
print("=" * 80)

print("\n1️⃣ METRIC CHOSEN:")
print("   Arm angle (shoulder-elbow-wrist angle)")

print("\n2️⃣ FRAMES USED:")
for data in results_data:
    print(f"   - {data['frame']} (Frame {data['frame_num']})")

print("\n3️⃣ NUMERICAL VALUES:")
print("\n   Right Arm Angles:")
for data in results_data:
    print(f"   - Frame {data['frame_num']}: {data['right_angle']:.1f}°")

print("\n   Left Arm Angles:")
for data in results_data:
    print(f"   - Frame {data['frame_num']}: {data['left_angle']:.1f}°")

print("\n4️⃣ CALCULATION EXPLANATION:")
print("""
   I extracted the (x, y) coordinates of the shoulder, elbow, and wrist keypoints 
   from YOLOv8-Pose detection for each frame. Since each frame contained two views 
   of the same person, I selected the """ + PERSON_POSITION + """ person based on horizontal position 
   in the frame. I then computed two vectors: one from the elbow to the shoulder 
   and another from the elbow to the wrist. Using the dot product formula 
   (cos θ = (v1 · v2) / (|v1| × |v2|)), I calculated the cosine of the angle, 
   then applied the inverse cosine (arccos) to get the angle in radians, which 
   I converted to degrees.
""")

print("\n5️⃣ GEOMETRIC INTERPRETATION:")
if len(results_data) >= 2:
    min_angle = min(results_data, key=lambda x: x['right_angle'])
    max_angle = max(results_data, key=lambda x: x['right_angle'])
    angle_range = max_angle['right_angle'] - min_angle['right_angle']
    
    first_frame = results_data[0]
    last_frame = results_data[-1]
    total_change = last_frame['right_angle'] - first_frame['right_angle']
    
    print(f"""
   The right arm angle varied from {min_angle['right_angle']:.1f}° (Frame {min_angle['frame_num']}) 
   to {max_angle['right_angle']:.1f}° (Frame {max_angle['frame_num']}), showing a total range of 
   {angle_range:.1f}°. Between the first frame ({first_frame['right_angle']:.1f}°) and last frame 
   ({last_frame['right_angle']:.1f}°), the arm angle changed by {total_change:+.1f}°, indicating 
   the arm {'extended' if total_change > 0 else 'flexed'} throughout the swing motion. This geometric 
   change demonstrates the transition from the initial setup position through the complete swing arc.
""")

print("\n✅ Output saved:")
print("   - Pose-overlaid images: 'pose_output/' folder")
print("   - Angle-annotated images: 'angle_output/' folder")

# ============================================================================
# BONUS: Export data to CSV for further analysis
# ============================================================================

print("\n" + "=" * 80)
print("📊 EXPORTING DATA TO CSV")
print("=" * 80)

# ⚠️ FIXED: Save CSV in angle_output folder
csv_file = os.path.join("angle_output", "arm_angles_data.csv")

try:
    with open(csv_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Frame_Number', 'Frame_File', 'Right_Arm_Angle', 'Left_Arm_Angle', 
                         'Right_Arm_Change', 'Left_Arm_Change'])
        
        for idx, data in enumerate(results_data):
            right_change = data['right_angle'] - results_data[idx-1]['right_angle'] if idx > 0 else 0
            left_change = data['left_angle'] - results_data[idx-1]['left_angle'] if idx > 0 else 0
            
            writer.writerow([
                data['frame_num'],
                data['frame'],
                f"{data['right_angle']:.1f}",
                f"{data['left_angle']:.1f}",
                f"{right_change:+.1f}",
                f"{left_change:+.1f}"
            ])
    
    print(f"✅ CSV file successfully saved: {os.path.abspath(csv_file)}")
    print(f"   Location: {csv_file}")
    
except Exception as e:
    print(f"❌ Error saving CSV: {e}")

print("=" * 80)
print("\n🎉 ANALYSIS COMPLETE!")
print("=" * 80)


GOLF SWING POSE ANALYSIS - Tiger Woods
📍 Analyzing: LEFT person in frame

Frame 1.png:
  Detected 2 person(s)
  Selected person at index 0 (left position)
  Average keypoint confidence: 0.938
  ✓ Saved pose overlay

Frame 2.png:
  Detected 2 person(s)
  Selected person at index 0 (left position)
  Average keypoint confidence: 0.944
  ✓ Saved pose overlay

Frame 3.png:
  Detected 2 person(s)
  Selected person at index 0 (left position)
  Average keypoint confidence: 0.933
  ✓ Saved pose overlay

Frame 4.png:
  Detected 2 person(s)
  Selected person at index 1 (left position)
  Average keypoint confidence: 0.844
  ✓ Saved pose overlay

Frame 5.png:
  Detected 2 person(s)
  Selected person at index 1 (left position)
  Average keypoint confidence: 0.852
  ✓ Saved pose overlay

Frame 6.png:
  Detected 2 person(s)
  Selected person at index 0 (left position)
  Average keypoint confidence: 0.858
  ✓ Saved pose overlay

Frame 7.png:
  Detected 2 person(s)
  Selected person at index 1 (left pos